<a href="https://colab.research.google.com/github/vanshitabansal/Bounding_box_generation/blob/main/bounding_box_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import cv2
from glob import glob
from tqdm import tqdm 
import csv
from pathlib import Path
from skimage.measure import label, regionprops, find_contours

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get all the images (.png) using glob
benign_images=glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/dataset/benign","*.png"))
malignant_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/dataset/malignant","*.png"))
normal_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/dataset/normal","*.png"))

# check if the path exist or not, if not create it to save the output
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign")
create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant")
create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal")

# Loop over the dataset to resize the images if required
for x in benign_images: 
    name = x.split("/")[-1].split(".")[0]
    image = Image.open(x)
    img=cv2.imread(x)
    img = cv2.resize(img, (256,256))
    im = Image.fromarray(img)
    im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign/{name}.png")

for x in malignant_images:
    name = x.split("/")[-1].split(".")[0]
    image = Image.open(x)
    img=cv2.imread(x)
    img = cv2.resize(img, (256,256))
    im = Image.fromarray(img)
    im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant/{name}.png")

for x in normal_images:
    name = x.split("/")[-1].split(".")[0]
    image = Image.open(x)
    img=cv2.imread(x)
    img = cv2.resize(img, (256,256))
    im = Image.fromarray(img)
    im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal/{name}.png")

In [ ]:
#  To get only boundaries from the mask
def mask_to_border(mask):
    h , w = mask.shape

    #  creating blank image of same size
    border = np.zeros ((h , w))

    contours = find_contours(mask, 128)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255 
    
    return border

def mask_to_bbox(mask):
    bboxes = []

    border = mask_to_border(mask)
    lbl = label(border)
    props = regionprops(lbl)

    for prop in props:
        x1 = prop.bbox[1]
        y1 = prop.bbox[0]

        x2 = prop.bbox[3]
        y2 = prop.bbox[2]

        bboxes.append([x1,y1,x2,y2])

    return bboxes

# field names for csv file
fields = ['image_id', 'Width', 'Height', 'bbox'] 

# name of csv file 
filename = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/train/train.csv"
    


# main function
if __name__ == "__main__":
    # load the dataset
    benign_images =glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign","*).png"))
    benign_mask = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign","*mask*.png"))
    malignant_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant","*).png"))
    malignant_mask = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant","*mask*.png"))
    normal_images = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal","*).png"))
    normal_mask = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal","*mask*.png"))

    
    # Create a folder to save images
    create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/benign")
    create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/malignant")
    create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/normal")

    # Loop over the benign dataset
    rows=[[]]
    for x in benign_images:
        name = x.split("/")[-1].split(".")[0]
        bboxes = []
        flag = 0

        # Read the image
        xx = cv2.imread(x,cv2.IMREAD_COLOR)
        for y in benign_mask:
             name2 = y.split("/")[-1].split(".")[0]
             if(name2.startswith(name)):

                # Read the mask 
                yy = cv2.imread(y,cv2.IMREAD_GRAYSCALE)
                
                # Detecting bounding boxes
                bboxes=mask_to_bbox(yy)

                # Marking bounding boxes in image
                coordinates=[]
                for bbox in bboxes:
                    xx = cv2.rectangle(xx,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),2)
                    coordinates.extend([bbox[0],bbox[1],bbox[2],bbox[3]])
                    rows.append([name,256,256,coordinates])
                    # Save the images
                    cv2.imwrite(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/benign/{name}.png",xx)
                # print(coordinates)
                
    # print(rows)
    # print(len(rows))
    # Loop over the malignant dataset
    for x in malignant_images:
        name = x.split("/")[-1].split(".")[0]
        bboxes = []
        flag = 0

        # Read the image
        xx = cv2.imread(x,cv2.IMREAD_COLOR)
        for y in malignant_mask:
             name2 = y.split("/")[-1].split(".")[0]
             if(name2.startswith(name)):

                # Read the mask 
                yy = cv2.imread(y,cv2.IMREAD_GRAYSCALE)
                
                # Detecting bounding boxes
                bboxes=mask_to_bbox(yy)

                # Marking bounding boxes in image
                coordinates=[]
                for bbox in bboxes:
                    xx = cv2.rectangle(xx,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),2)
                    coordinates.extend([bbox[0],bbox[1],bbox[2],bbox[3]])
                    rows.append([name,256,256,coordinates])
                    # Save the images
                    cv2.imwrite(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/malignant/{name}.png",xx)

    
   
    
    for x,y in tqdm(zip(normal_images,normal_mask),total=len(normal_mask)):

        # Extracting the name of image and also removing .png
        name = y.split("/")[-1].split(".")[0]
        # Read the image and mask
        x = cv2.imread(x,cv2.IMREAD_COLOR)
        y= cv2.imread(y,cv2.IMREAD_GRAYSCALE)

        # Detecting bounding boxes 
        bboxes = mask_to_bbox(y)

        # Marking bounding boxes in image
        coordinates=[]
        for bbox in bboxes:
            x = cv2.rectangle(x,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),2)
            coordinates.extend([bbox[0],bbox[1],bbox[2],bbox[3]])
            rows.append([name,256,256,coordinates])
        # Save the images
        cv2.imwrite(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Results/normal/{name}.png",x)

print(rows)
# writing to csv file 
with open(filename, 'w',newline='') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)



100%|██████████| 133/133 [00:03<00:00, 41.37it/s]

[[], ['benign (106)', 256, 256, [71, 61, 113, 102]], ['benign (10)', 256, 256, [8, 28, 87, 116]], ['benign (108)', 256, 256, [181, 43, 246, 91]], ['benign (102)', 256, 256, [106, 21, 182, 75]], ['benign (104)', 256, 256, [198, 26, 228, 50]], ['benign (109)', 256, 256, [23, 46, 125, 137]], ['benign (11)', 256, 256, [135, 57, 163, 74]], ['benign (107)', 256, 256, [125, 95, 170, 136]], ['benign (100)', 256, 256, [21, 57, 78, 86]], ['benign (100)', 256, 256, [156, 67, 237, 120]], ['benign (103)', 256, 256, [8, 68, 123, 151]], ['benign (101)', 256, 256, [198, 92, 244, 121]], ['benign (12)', 256, 256, [129, 79, 159, 100]], ['benign (105)', 256, 256, [121, 67, 159, 108]], ['benign (135)', 256, 256, [66, 25, 111, 73]], ['benign (123)', 256, 256, [114, 47, 203, 105]], ['benign (129)', 256, 256, [61, 69, 107, 98]], ['benign (134)', 256, 256, [149, 43, 189, 63]], ['benign (114)', 256, 256, [66, 40, 142, 106]], ['benign (121)', 256, 256, [100, 18, 172, 71]], ['benign (13)', 256, 256, [68, 47, 203,

#code to generate DSMI.csv

In [ ]:

# images=glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Data/Images","*.png"))
# fimages = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Data/foreground","*.png"))
# bimages = glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Data/background","*.png"))
# import os
# import numpy as np
# import cv2
# import csv
# # check if the path exist or not, if not create it to save the output
# def create_dir(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

# # field names for csv file
# fields = ['image_id','f_id','b_id', 'Width', 'Height', 'bbox'] 

# # name of csv file 
# filename = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Data/DSMI_t.csv"
# # create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/benign")
# # create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/malignant")
# # create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Modified/normal")
# l1=[]
# l2=[]
# l3=[]
# # Loop over the benign dataset
# for x in images:
#     name = x.split("/")[-1].split(".")[0]
#     l1.append(name)

# for x in fimages:
#     name = x.split("/")[-1].split(".")[0]
#     l2.append(name)


# for x in bimages:
#     name = x.split("/")[-1].split(".")[0]
#     l3.append(name)

# l1.sort()
# l2.sort()
# l3.sort()
# print(l1)
# print(l2)
# print(l3)
# print(len(l1))
# print(len(l2))
# print(len(l3))
# rows=[[]]
# for i in range(len(l1)):
#   rows.append([l1[i],l2[i],l3[i],256,256,"coordinates"])

# with open(filename, 'w',newline='') as csvfile: 
#     # creating a csv writer object 
#     csvwriter = csv.writer(csvfile) 
        
#     # writing the fields 
#     csvwriter.writerow(fields) 
        
#     # writing the data rows 
#     csvwriter.writerows(rows)

['benign (1)', 'benign (10)', 'benign (100)', 'benign (100)_1)', 'benign (101)', 'benign (102)', 'benign (103)', 'benign (104)', 'benign (105)', 'benign (106)', 'benign (107)', 'benign (108)', 'benign (109)', 'benign (11)', 'benign (110)', 'benign (111)', 'benign (112)', 'benign (113)', 'benign (114)', 'benign (115)', 'benign (116)', 'benign (117)', 'benign (118)', 'benign (119)', 'benign (12)', 'benign (120)', 'benign (121)', 'benign (122)', 'benign (123)', 'benign (124)', 'benign (125)', 'benign (126)', 'benign (127)', 'benign (128)', 'benign (129)', 'benign (13)', 'benign (130)', 'benign (131)', 'benign (132)', 'benign (133)', 'benign (134)', 'benign (135)', 'benign (136)', 'benign (137)', 'benign (138)', 'benign (139)', 'benign (14)', 'benign (140)', 'benign (141)', 'benign (142)', 'benign (143)', 'benign (144)', 'benign (145)', 'benign (146)', 'benign (147)', 'benign (148)', 'benign (149)', 'benign (15)', 'benign (150)', 'benign (151)', 'benign (152)', 'benign (153)', 'benign (154

# code to fix foreground and image in DSMI

In [ ]:
import csv
import os
import numpy as np
import cv2
import torch
import torchvision
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from glob import glob
from tqdm import tqdm 
from pathlib import Path
img_dir = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/Images1"
img_dir2 = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/foreground"
img_dir3 = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/background"
csv_path = "/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/train/Validation/DSMI.csv"
images=glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/Images","*.png"))
l=[]
# check if the path exist or not, if not create it to save the output
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/background2")
create_dir("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/foreground2")
for x in images:
    name = x.split("/")[-1].split(".")[0]
    l.append(name)

images=glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/foreground","*.png"))
for x in images:
    name = x.split("/")[-1].split(".")[0]
    for i in range(len(l)):
      if name.startswith(l[i]):
        if name.endswith("mask"):
          old_path = os.path.join(img_dir2, name+'.png')
          image = Image.open(old_path)
          img=cv2.imread(old_path)
          im = Image.fromarray(img)
          print(name," ",l[i])
          im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/foreground2/{l[i]}.png")
for x in images:
    name = x.split("/")[-1].split(".")[0]
    for i in range(len(l)):
      if(l[i].find('_')>=0):
        prefixl=l[i].split("_")[0]
        # print(prefixl," ",l[i])
        if name.startswith(prefixl):
          if name.endswith("mask")==False:
            old_path = os.path.join(img_dir2, name+'.png')
            image = Image.open(old_path)
            img=cv2.imread(old_path)
            im = Image.fromarray(img)
            print(name," ",l[i])
            im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/foreground2/{l[i]}.png")

images=glob(os.path.join("/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/background","*.png"))
for x in images:
    name = x.split("/")[-1].split(".")[0]
    for i in range(len(l)):
      if name.startswith(l[i]):
        if name.endswith("mask"):
          old_path = os.path.join(img_dir3, name+'.png')
          image = Image.open(old_path)
          img=cv2.imread(old_path)
          im = Image.fromarray(img)
          print(name," ",l[i])
          im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/background2/{l[i]}.png")
for x in images:
    name = x.split("/")[-1].split(".")[0]
    for i in range(len(l)):
      if(l[i].find('_')>=0):
        prefixl=l[i].split("_")[0]
        # print(prefixl," ",l[i])
        if name.startswith(prefixl):
          if name.endswith("mask")==False:
            old_path = os.path.join(img_dir3, name+'.png')
            image = Image.open(old_path)
            img=cv2.imread(old_path)
            im = Image.fromarray(img)
            print(name," ",l[i])
            im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/Validation/background2/{l[i]}.png")
# with open(csv_path, "r") as f:
#     reader = csv.reader(f)
#     next(reader)  # Skip header row
#     # print(reader)
#     for row in reader:
#         # print(row)
#         old_name = row[0]
#         new_name = row[6]
#         old_path = os.path.join(img_dir, old_name+'.png')
#         print(old_name," ",new_name)
#         if os.path.exists(old_path) and old_name.startswith("b"):
#           print(old_name," ",new_name)
#           image = Image.open(old_path)
#           img=cv2.imread(old_path)
#           im = Image.fromarray(img)
#           im.save(f"/content/drive/MyDrive/Implementation of Detectron2 based faster rcnn/test_frcnn/benign/{new_name}.png")
#         # old_path = os.path.join(img_dir, old_name+'.png')
#         # if os.path.exists(old_path):
#         #   print(old_name," ",new_name)
#         #   new_path = os.path.join(img_dir2, new_name+'.png')
#         #   os.rename(old_path, new_path)

benign (8)_mask   benign (8)
benign (9)_mask   benign (9)
benign (39)_mask   benign (39)
benign (4)_mask   benign (4)
benign (50)_mask   benign (50)
benign (45)_mask   benign (45)
benign (46)_mask   benign (46)
benign (6)_mask   benign (6)
benign (38)_mask   benign (38)
benign (36)_mask   benign (36)
benign (40)_mask   benign (40)
benign (35)_mask   benign (35)
benign (42)_mask   benign (42)
benign (43)_mask   benign (43)
benign (44)_mask   benign (44)
benign (37)_mask   benign (37)
benign (5)_mask   benign (5)
benign (7)_mask   benign (7)
benign (47)_mask   benign (47)
benign (34)_mask   benign (34)
benign (48)_mask   benign (48)
benign (49)_mask   benign (49)
benign (41)_mask   benign (41)
benign (27)_mask   benign (27)
benign (26)_mask   benign (26)
benign (23)_mask   benign (23)
benign (25)_mask   benign (25)
benign (30)_mask   benign (30)
benign (31)_mask   benign (31)
benign (32)_mask   benign (32)
benign (33)_mask   benign (33)
benign (22)_mask   benign (22)
benign (28)_mask   b